<a href="https://colab.research.google.com/github/MadHatter999/ALGO_LiDAR3.0/blob/main/Feature_Extraction_Gray_Level_Co_occurrence_Matrix_(GLCM)_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Nous utiliserons la bibliothèque 'os' pour lire les noms de fichiers des dossiers et la bibliothèque 'cv2' (OpenCV) pour faire le traitement d'image**

In [71]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [72]:
from PIL import Image , ImageDraw

In [73]:
def NoiseGenerator(height, width,number):#generer une image bruit
    for i in range (number):
        array = np.random.randint(255, size=(height, width),dtype=np.uint8)
        image = Image.fromarray(array)
        #image.show()
        nom=str(i)+".png" 
        image.save("/noise"+nom)

In [74]:
def decouper(image):#corp l'images en des morceaux 50*50
    width, height = image.size
    if height%50==0:
        i_max=height//50
    else:
        i_max=height//50+1

    if width%50==0:
        j_max=width//50
    else:
        j_max=width//50+1

#im1 = im.crop((left, top, right, bottom)) 
    for i in range(i_max):
        for j in range(j_max):
            if i==i_max:
                if j==j_max:
                    croped= image.crop((50*j+1, 50*i+1, width,height)) 
                else:
                    croped= image.crop((50*j+1, 50*i+1,50*(j+1)+1 ,height)) 
            else:
                if j==j_max:
                    croped= image.crop((50*j+1, 50*i+1, width,50*(i+1)+1)) 
                else:
                    croped= image.crop((50*j+1, 50*i+1,50*(j+1)+1 ,50*(i+1)+1)) 
            nom=str(j+1)+"."+str(i+1)+".png"
            croped.save("MyDrive/images de bruit/"+nom)

In [75]:
def ecrire(text):#retourner une image contenant la valeur d'un des indices
    img = Image.new('RGB', (50,50), color = (255,255,255))
    d = ImageDraw.Draw(img)
    d.text((1,20), text, fill=(0,0,0))
    return(img)


In [76]:
%cd ..
image=Image.open('home/dice.png')
%cd /gdrive
decouper(image)

/
/gdrive


In [77]:
import numpy as np 
import cv2 
import os
import re

# -------------------- Utility function ------------------------
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder 
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")
        

# -------------------- Load Dataset ------------------------
 
dataset_dir = '/gdrive'

imgs = [] #list image matrix 
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]
            
            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
            
            imgs.append(resize)
            labels.append(normalize_label(os.path.splitext(filename)[0]))
            descs.append(normalize_desc(folder, sub_folder))
            
            print_progress(i, len_sub_folder, folder, sub_folder, filename)

**2) L'ensemble de données d'images de feuilles sera stocké dans la liste 'imgs' et les noms de feuilles seront stockés dans la liste 'labels'. Nous pouvons prévisualiser l'une des images de la manière suivante**

In [78]:
from google.colab.patches import cv2_imshow
cv2_imshow(imgs[0])

**3) Ensuite, nous pouvons trouver la matrice GLCM et la métrique de texture globale de l'image de la feuille en utilisant les fonctions greycomatrix() et greycoprops() dans la bibliothèque Scikit-Image.**

In [79]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels): 
    glcm_all_agls.append(
            calc_glcm_all_agls(img, 
                                label, 
                                props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")

**4) Ensuite, nous changeons le format de données glcm_features de liste en Dataframe à l'aide de la bibliothèque pandas**

In [109]:
import pandas as pd 

# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls, 
                      columns = columns)
glcm_df['label'] = glcm_df['label']
sorted_df = glcm_df.sort_values(by=['label'], ascending=True)
df=sorted_df.reset_index(drop=True)
#df.columns
df["dissimilarity_0"][0]

2.166666666666667

In [110]:

%cd ..
%cd "/home"

/
/home


In [111]:
!pwd

/home


In [112]:
width, height = image.size
if height%50==0:
    i_max=height//50
else:
    i_max=height//50+1

if width%50==0:
    j_max=width//50
else:
    j_max=width//50+1
result = Image.new('RGB', (j_max*50,i_max*50))
indexe=0
for i in range(i_max):
    for j in range(j_max):
        text=str(df["dissimilarity_0"][i*(j_max-1)+j])
        im=ecrire(text)
        result.paste(im,(j*50,i*50))
nom=df.columns[indexe]+'.png'
result.save(nom)




In [113]:
df

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,2.166667,3.1250,2.416667,1.3750,0.042945,-0.457021,0.046429,0.387826,0.307240,0.236693,...,2.7500,0.048611,0.042969,0.036458,0.058594,0.220479,0.207289,0.190941,0.242061,1.1
1,0.291667,39.8750,53.416667,40.2500,0.999844,-0.142849,-0.199987,-0.142854,0.906618,0.750010,...,6480.3750,0.768229,0.570312,0.458333,0.574219,0.876487,0.755190,0.677003,0.757772,1.10
2,2.541667,4.2500,3.250000,1.7500,0.130176,-0.557692,-0.253968,0.136364,0.240851,0.087443,...,4.7500,0.035590,0.042969,0.026042,0.044922,0.188654,0.207289,0.161374,0.211948,1.2
3,1.750000,3.1250,3.041667,1.8125,0.404878,-0.167477,-0.291720,-0.019269,0.427338,0.182975,...,5.0625,0.035590,0.042969,0.032118,0.060547,0.188654,0.207289,0.179215,0.246063,1.3
4,2.583333,3.3750,2.458333,1.5625,0.012161,-0.450187,-0.060413,0.397113,0.300772,0.171221,...,4.8125,0.035590,0.056641,0.051215,0.072266,0.188654,0.237993,0.226308,0.268823,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,1.666667,4.0000,5.541667,4.8125,0.745889,-0.745283,-0.799031,-0.419396,0.379902,0.189609,...,25.8125,0.032118,0.060547,0.042535,0.048828,0.179215,0.246063,0.206239,0.220971,9.5
146,6.875000,12.3125,8.625000,1.5000,0.142944,-0.714912,-0.199625,0.802107,0.042412,0.007155,...,3.3750,0.020833,0.031250,0.026042,0.039062,0.144338,0.176777,0.161374,0.197642,9.6
147,1.333333,2.1250,1.541667,0.9375,-0.016949,-0.258427,-0.065140,0.363033,0.521569,0.311029,...,1.3125,0.077257,0.066406,0.066840,0.093750,0.277951,0.257694,0.258535,0.306186,9.7
148,1.625000,1.6250,1.375000,1.6250,-0.184061,-0.444444,0.179583,-0.192308,0.475622,0.578895,...,3.8750,0.065104,0.119141,0.059896,0.064453,0.255155,0.345168,0.244736,0.253876,9.8
